In [1]:
import functools

In [2]:
import numpy as np

In [3]:
import math

In [4]:
import tensorflow as tf

In [5]:
from tensorflow.python import debug as tf_debug

In [6]:
from enum import IntEnum

In [7]:
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


In [8]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
training_data, testing_data = tf.keras.datasets.fashion_mnist.load_data()

In [9]:
number_of_classes = np.max(testing_data[1]) + 1
number_of_input_channels = 1

In [10]:
batch_size = 32

In [11]:
number_of_layers = 3 # 1000

In [12]:
lod_channels = [8, 8, 8]

In [13]:
def get_number_of_samples_per_fiber():
    return 3

In [14]:
OffsetIndex = IntEnum("OffsetIndex", names=["OFFSET_X", "OFFSET_Y"], start=0)
ChannelIndex = IntEnum("ChannelIndex", names=["TARGET_CHANNEL", "SOURCE_CHANNEL"], start=0)

In [15]:
root_variable_scope = tf.get_variable_scope()

In [16]:
def fiber(number_of_target_channels, number_of_source_channels, source_layer):
#     number_of_source_channels = source_layer[3]
    def generate_channel_indices():
        for target_channel_index in range(number_of_target_channels):
            for source_channel_index in range(number_of_source_channels):
                number_of_samples_per_fiber = get_number_of_samples_per_fiber()
                for fiber_index in range(number_of_samples_per_fiber):
                    yield (target_channel_index, source_channel_index)
                    
    def generate_target_channel_indices():
        for target_channel_index, source_channel_index in generate_channel_indices():
            yield target_channel_index

    def generate_source_channel_indices():
        for target_channel_index, source_channel_index in generate_channel_indices():
            yield source_channel_index

    target_channel_indices = tf.get_variable(name="target_channel_indices", initializer=tuple(generate_target_channel_indices()), trainable=False)
    source_channel_indices = tf.get_variable(name="source_channel_indices", initializer=tuple(generate_source_channel_indices()), trainable=False)
    
    number_of_sample_points = source_channel_indices.shape[0]
    offset_xy = tf.get_variable(name="offset_xy", trainable=False, initializer=tf.random_normal_initializer(stddev=3), dtype=tf.float32, shape=(number_of_sample_points, len(OffsetIndex.__members__)))
    weight = tf.get_variable(name="weight", initializer=tf.glorot_normal_initializer(), dtype=tf.float32, shape=[number_of_sample_points, 1, 1, 1])
    with tf.contrib.compiler.jit.experimental_jit_scope():
        return tf.transpose(
            tf.unsorted_segment_sum(
                weight * tf.contrib.image.translate(
                    tf.gather(
                        tf.transpose(
                            source_layer,
                            perm=(3, 1, 2, 0)    
                        ),
                        indices=source_channel_indices
                    ),
                    translations=offset_xy
                ),
                segment_ids=target_channel_indices,
                num_segments=number_of_target_channels
            ),
            perm=(3, 1, 2, 0)  
        )

In [17]:
def model_fn(features, labels, mode, params, config):
    input_shape = features.shape
    def lod_size(i):
        scale = 2 ** i
        return input_shape[1] // tf.Dimension(scale), input_shape[2] // tf.Dimension(scale)

    @functools.lru_cache(maxsize=None)
    def input_layer(current_lod):
        with tf.variable_scope(root_variable_scope), tf.variable_scope(f"scale_{current_lod}"), tf.name_scope(f"scale_{current_lod}/"):
            if current_lod == 0:
                return tf.expand_dims(tf.cast(features, tf.float32), axis=3) / 255.0
            else:
                # Pooling
    #             tf.nn.avg_pool(
    #                 value=layer(i, lod, current_lod - 1, channel),
    #                 ksize=[1, 2, 2, 1],
    #                 strides=[1, 1, 1, 1],
    #                 padding='SAME'
    #             )
                return tf.image.resize_bilinear(
                    images=input_layer(current_lod - 1),
                    size=lod_size(current_lod),
                    align_corners=False,
                )
    @functools.lru_cache(maxsize=None)
    def layer(target_layer_index, preferred_lod, current_lod):
        """Return a list of tensor of shape batch_size ⨉ lod_size(height) ⨉ lod_size(width) ⨉ channel

        The list size is sum(lod_channels)
        """
        with tf.variable_scope(root_variable_scope), tf.variable_scope(f"layer_{target_layer_index}"), tf.name_scope(f"layer_{target_layer_index}/"):
            if current_lod == preferred_lod:
                number_of_target_channels = lod_channels[preferred_lod]
                with tf.variable_scope(f"lod_{preferred_lod}"):
                    def source_layer_mapper(source_layer_index):
                        with tf.variable_scope(f"weighted_layer_{source_layer_index}"):
                            def source_lod_mapper(source_lod, number_of_source_channels):
                                with tf.variable_scope(f"weighted_lod_{source_lod}"):
                                    return fiber(
                                        number_of_target_channels=number_of_target_channels,
                                        number_of_source_channels=number_of_source_channels,
                                        source_layer=layer(source_layer_index, source_lod, current_lod)
                                    )
                            return sum(source_lod_mapper(source_lod, number_of_source_channels)
                                       for source_lod, number_of_source_channels in enumerate(lod_channels))
                    channel_scores = sum(map(source_layer_mapper, range(target_layer_index)))
                    bias = tf.get_variable(name="bias", initializer=tf.zeros_initializer(), dtype=tf.float32, shape=(1, 1, 1, number_of_target_channels))
                    with tf.variable_scope("weighted_input"):
                        input_fiber = fiber(number_of_target_channels=number_of_target_channels,
                                            number_of_source_channels=number_of_input_channels,
                                            source_layer=input_layer(current_lod))
                        return bias + input_fiber + channel_scores
            elif preferred_lod < current_lod:
                # Pooling
    #             tf.nn.avg_pool(
    #                 value=layer(i, lod, current_lod - 1, channel),
    #                 ksize=[1, 2, 2, 1],
    #                 strides=[1, 1, 1, 1],
    #                 padding='SAME'
    #             )
                return tf.image.resize_bilinear(
                    images=layer(target_layer_index, preferred_lod, current_lod - 1),
                    size=lod_size(current_lod),
                    align_corners=False,
                )
            elif preferred_lod > current_lod:
                # Unpooling
                return tf.image.resize_bilinear(
                    images=layer(target_layer_index, preferred_lod, current_lod + 1),
                    size=lod_size(current_lod),
                    align_corners=False,
                )
    lowest_lod = len(lod_channels) - 1
    scores = tf.nn.xw_plus_b(
        x=tf.reduce_mean(layer(number_of_layers - 1, lowest_lod, lowest_lod), axis=(1, 2)),
        weights=tf.get_variable(name="dense_weight", initializer=tf.random_normal_initializer(), dtype=tf.float32, shape=(lod_channels[lowest_lod], number_of_classes)),
        biases=tf.get_variable(name="dense_bias", initializer=tf.zeros_initializer(), dtype=tf.float32, shape=number_of_classes)
    )
    probabilities = tf.nn.softmax(logits=scores)
    predicted_classes = tf.argmax(scores, 1)
    predictions = {
        'probabilities' : probabilities,
        'scores': scores,
        'class': predicted_classes,
    }
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)
    }
    loss = tf.losses.softmax_cross_entropy(logits=scores, onehot_labels=tf.one_hot(labels, number_of_classes))
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops
    )

In [18]:
run_config = tf.estimator.RunConfig(
    model_dir=f"models/two-indicies-offnet{number_of_layers}",
    session_config=tf.ConfigProto(
        graph_options=tf.GraphOptions(
            optimizer_options=tf.OptimizerOptions(
                global_jit_level=tf.OptimizerOptions.ON_2,
                do_function_inlining=True,
                do_constant_folding=True,
                do_common_subexpression_elimination=True,
            )
        )
    )
)

In [19]:
estimator = tf.estimator.Estimator(model_fn, config=run_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/two-indicies-offnet3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': graph_options {
  optimizer_options {
    do_common_subexpression_elimination: true
    do_constant_folding: true
    do_function_inlining: true
    global_jit_level: ON_2
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6ac731f630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
def training_dataset():
    return tf.data.Dataset.from_tensor_slices(training_data).shuffle(1000).repeat().batch(batch_size)

In [21]:
def testing_dataset():
    return tf.data.Dataset.from_tensor_slices(testing_data).batch(batch_size)

In [22]:
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
# estimator.train(training_dataset,hooks=[hook])

In [23]:
tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(training_dataset),
    eval_spec=tf.estimator.EvalSpec(testing_dataset)
)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.


/mnt/project-root/envs/default/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/two-indicies-offnet3/model.ckpt.
INFO:tensorflow:loss = 2.3047924, step = 1
INFO:tensorflow:global_step/sec: 1.92804
INFO:tensorflow:loss = 2.2841187, step = 101 (51.867 sec)
INFO:tensorflow:global_step/sec: 1.96796
INFO:tensorflow:loss = 2.2569315, step = 201 (50.814 sec)
INFO:tensorflow:global_step/sec: 1.9696
INFO:tensorflow:loss = 2.282398, step = 301 (50.772 sec)


KeyboardInterrupt: 